[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/emiletimothy/Caltech-CS155-2023/blob/main/set5/set5_prob2.ipynb)


# Problem 2

In [102]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [103]:
movies = pd.read_csv("../movies.csv")
data = pd.read_csv("../data.csv")
print(data)

        User ID  Movie ID  Rating
0             0        88     4.0
1             0       545     3.5
2             0         7     3.0
3             0       401     4.0
4             0      1230     0.5
...         ...       ...     ...
121496      991       681     1.0
121497      991       736     3.5
121498      991       162     3.0
121499      991      1242     3.5
121500      991       321     4.5

[121501 rows x 3 columns]


In [104]:
# #Typecast columns User ID and Movie ID of data as integers
# data['User ID'] = data['User ID'].astype(int)
# data['Movie ID'] = data['Movie ID'].astype(int)
# Needs format: matrix Y containing rows (i, j, Y_ij) where Y_ij is user i's rating on movie j
Y = np.array(data)
print(Y)
print(len(Y))
random.shuffle(Y)
print(Y)

[[   0.    88.     4. ]
 [   0.   545.     3.5]
 [   0.     7.     3. ]
 ...
 [ 991.   162.     3. ]
 [ 991.  1242.     3.5]
 [ 991.   321.     4.5]]
121501
[[   0.    88.     4. ]
 [   0.    88.     4. ]
 [   0.   545.     3.5]
 ...
 [ 382.  1281.     4. ]
 [ 193.   344.     4.5]
 [ 909.   128.     4. ]]


In [105]:
from numpy.random.mtrand import gamma
def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    sum = Vj*(Yij - np.dot(Ui, Vj))
    grad = reg*Ui-sum
    return eta*grad

def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    sum = Ui*(Yij - np.dot(Ui, Vj))
    grad = reg*Vj-sum
    return eta*grad

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    sum = 0
    for trip in Y:
      i = int(trip[0])
      j = int(trip[1])
      Yij = trip[2]
      # print("i: "+str(i)+" j: "+str(j))
      # print("len U: "+str(len(U))+" len V: "+str(len(V)))
      sum += (Yij-np.dot(U[i], V[j]))**2
      
    return float(sum/(2*len(Y)))



def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    U = np.random.uniform(low=-0.5, high=0.5, size=(M,K))
    V = np.random.uniform(low=-0.5, high=0.5, size=(N,K))
    e=0
    mse_0 = get_err(U, V, Y, reg)
    prev_mse = mse_0
    cur_mse = mse_0
    first_mse = 0
    first = True
    while e<max_epochs:
      np.random.shuffle(Y)
      for trip in Y:
        i = int(trip[0])
        j = int(trip[1])
        Yij = trip[2]
        U[i] -= grad_U(U[i], Yij, V[j], reg, eta)
        V[j] -= grad_V(V[j], Yij, U[i], reg, eta)

      cur_mse = get_err(U, V, Y, reg)
      if(first):
        first_mse = mse_0-cur_mse
        first = False
      
      if (prev_mse-cur_mse)<=(first_mse*eps):
        break

      prev_mse = cur_mse
      e += 1

    return U, V, cur_mse

In [106]:
Y_train = Y[0:109351]
Y_test = Y[109351:121501]

In [107]:

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
M += 1
N += 1
print("Factorizing with ", M, " users, ", N, " movies.")
K = 20

reg = 0.0
eta = 0.03 # learning rate
E_in = []
E_out = []

# Use to compute Ein and Eout

U,V, err = train_model(M, N, K, eta, reg, Y_train)

# E_in.append(err)
# E_out.append(get_err(U, V, Y_test))

# plt.plot(K, E_in, label='$E_{in}$')
# plt.plot(K, E_out, label='$E_{out}$')
# plt.title('Error vs. K')
# plt.xlabel('K')
# plt.ylabel('Error')
# plt.legend()
# plt.savefig('2d.png')

Factorizing with  992  users,  1500  movies.


In [109]:
print(f"Testing error: {get_err(U, V, Y_test)}")


Testing error: 0.8352949063664049


In [112]:
#s singular value decomposition of V
A, s, B = np.linalg.svd(V, full_matrices=False)
U_2d = np.dot(A[1:2], np.transpose(U))
V_2d = np.dot(A[1:2], np.transpose(V))

print(U_2d)
print(V_2d)

[[-2.14263317e-03 -6.39603691e-02 -5.90336782e-02 -7.76047912e-02
  -6.57627811e-02 -2.16392508e-01  1.00685214e-01 -8.19235616e-02
  -1.35751742e-01 -6.46197885e-02 -1.03125824e-01 -5.81424206e-02
   1.85191533e-02 -1.11938908e-01 -7.63666441e-02  8.95843593e-03
  -5.06391899e-02 -6.43614016e-02  7.23590556e-02 -5.76003787e-02
  -4.45710406e-02 -1.39056099e-01 -9.96465682e-02 -4.21869383e-02
  -1.02501544e-01 -7.72970529e-02  3.60114787e-03 -5.08968418e-02
   4.06720835e-03 -6.01250642e-02 -4.39142152e-02  1.19366153e-01
  -6.51136032e-02 -7.66217898e-02 -2.33922136e-02 -9.60824584e-02
  -1.03980856e-01 -5.59592341e-02 -9.58008664e-02 -5.11531880e-02
  -3.75958689e-01 -5.26096880e-02 -1.18306868e-01 -1.10720222e-01
  -9.00610951e-02 -1.03721488e-01 -6.58784128e-02 -1.26871959e-01
  -8.20179846e-02 -1.74012651e-01  2.65057890e-02 -2.92914908e-02
  -1.03655608e-01 -1.60192352e-01 -1.12709070e-01 -8.75482343e-02
  -1.12358877e-02 -2.63437518e-02 -7.76999617e-02 -1.41607072e-01
  -5.51618

: 